In [3]:
!pip install scipy


In [5]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [6]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_womens_worldcup_historical_data.csv')
df_fixture= pd.read_csv('clean_fifa_womens_worldcup_fixture.csv')

In [8]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [21]:
#Calculating Team Strength

In [27]:
#split a df into df_home and df_away
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]


In [29]:
#renaming columns in this df_home
df_home=df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away=df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

In [31]:
#concadinating df_home and df_away to calculate the mean
df_team_strength= pd.concat([df_home , df_away], ignore_index =True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Argentina,0.555556,4.111111
Australia,1.461538,1.923077
Brazil,1.941176,1.176471
Cameroon,1.500000,1.500000
Canada,1.259259,1.925926
Chile,0.666667,1.666667
China,1.606061,0.969697
Chinese Taipei,0.500000,3.750000
Colombia,0.571429,1.285714


In [32]:
#Function for predict_points

In [45]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [34]:
#Testing function

In [51]:
#Testing with Group_a
predict_points('New Zealand','Switzerland')


(0.02521759848624812, 2.8035429088503583)

In [52]:
#predicting wold cup

In [64]:
#Group stage
df_fixture[:48]

,home,score,away,year
0,New Zealand,Match 1,Norway,2023
1,Philippines,Match 3,Switzerland,2023
2,New Zealand,Match 17,Philippines,2023
3,Switzerland,Match 18,Norway,2023
4,Switzerland,Match 33,New Zealand,2023
5,Norway,Match 34,Philippines,2023
6,Australia,Match 2,Republic of Ireland,2023
7,Nigeria,Match 4,Canada,2023
8,Canada,Match 19,Republic of Ireland,2023
9,Australia,Match 22,Nigeria,2023


In [65]:
#splitting the fixtures into grp stage,knockout,quarter,semi,finals
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [67]:
#running all the matches in the grp stage and updating grp table
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [74]:
#showing updated table
dict_table['Group G']

,Team,Pts
0,Italy,14.0
1,Sweden,14.0
2,Argentina,2.0
3,South Africa,2.0


In [75]:
##Knock-Out

In [76]:
df_fixture_knockout

,home,score,away,year
48,Winner Group A,Match 49,Runner-up Group C,2023
49,Winner Group C,Match 50,Runner-up Group A,2023
50,Winner Group E,Match 51,Runner-up Group G,2023
51,Winner Group G,Match 52,Runner-up Group E,2023
52,Winner Group D,Match 54,Runner-up Group B,2023
53,Winner Group B,Match 53,Runner-up Group D,2023
54,Winner Group H,Match 56,Runner-up Group F,2023
55,Winner Group F,Match 55,Runner-up Group H,2023


In [81]:
#updating knock out fixture:
   for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winner {group}':group_winner,
                                 f'Runner-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout 

,home,score,away,year,winner
48,Switzerland,Match 49,Spain,2023,?
49,Costa Rica,Match 50,Norway,2023,?
50,United States,Match 51,Sweden,2023,?
51,Italy,Match 52,Netherlands,2023,?
52,China,Match 54,Nigeria,2023,?
53,Canada,Match 53,England,2023,?
54,Germany,Match 56,Brazil,2023,?
55,France,Match 55,Colombia,2023,?


In [82]:
#predicting the knock out winners
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [85]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Switzerland,Match 49,Spain,2023,Switzerland
49,Costa Rica,Match 50,Norway,2023,Norway
50,United States,Match 51,Sweden,2023,United States
51,Italy,Match 52,Netherlands,2023,Italy
52,China,Match 54,Nigeria,2023,China
53,Canada,Match 53,England,2023,England
54,Germany,Match 56,Brazil,2023,Germany
55,France,Match 55,Colombia,2023,France


In [94]:
##predicting the quarter finals
df_fixture_quarter

,home,score,away,year,winner
56,Winner Match 49,Match 57,Winner Match 51,2023,?
57,Winner Match 50,Match 58,Winner Match 52,2023,?
58,Winner Match 53,Match 59,Winner Match 55,2023,?
59,Winner Match 54,Match 60,Winner Match 56,2023,?


In [97]:
#updating the quarter finals fixture
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winner {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [98]:
update_table(df_fixture_knockout,df_fixture_quarter)

,home,score,away,year,winner
56,Switzerland,Match 57,United States,2023,?
57,Norway,Match 58,Italy,2023,?
58,England,Match 59,France,2023,?
59,China,Match 60,Germany,2023,?


In [99]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Switzerland,Match 57,United States,2023,United States
57,Norway,Match 58,Italy,2023,Norway
58,England,Match 59,France,2023,France
59,China,Match 60,Germany,2023,Germany


In [100]:
#semifinals
update_table(df_fixture_quarter,df_fixture_semi)

,home,score,away,year,winner
60,United States,Match 61,Norway,2023,?
61,France,Match 62,Germany,2023,?


In [101]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,United States,Match 61,Norway,2023,United States
61,France,Match 62,Germany,2023,Germany


In [102]:
#THE WINNER OF THE 2023 WOMEN'S WORLD CUP :

In [103]:
update_table(df_fixture_semi,df_fixture_final)

,home,score,away,year,winner
62,Loser Match 61,Match 63,Loser Match 62,2023,?
63,United States,Match 64,Germany,2023,?


In [104]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Loser Match 61,Match 63,Loser Match 62,2023,Loser Match 62
63,United States,Match 64,Germany,2023,United States
